### Crawling list
- link
- img
- brand 
- origin
- title
- type
- price
- rating (integer) 
- optionTitle
- optionPrice (integer)
- replyConunt
- replyResult

In [875]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, NoSuchFrameException
from selenium.common.exceptions import TimeoutException
import requests
import time
time.sleep(1)
# 드라이버 로드

In [509]:
driver = wd.Chrome(executable_path='chromedriver.exe')

In [28]:
headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',    
    'accept-encoding':'false'
}

In [3]:
def getUrl(seed):
    driver.get(seed)
    html = bs(driver.page_source, 'lxml')
    links = driver.find_elements_by_css_selector('.pub_photo a')
    return [link.get_attribute('href') for link in links if link.get_attribute('href')]

### 분유

In [514]:
seed_milkPowder = 'http://www.11st.co.kr/browsing/BestSeller.tmall?method=getBestSellerMain&dispCtgrNo=1001345'

In [522]:
# requset 형식
link = getUrl(seed_milkPowder)
len(link)

100

In [913]:
def getInfo_milkPowder(url):

    driver.get(url)
    bs(driver.page_source, 'lxml')
    
    # crawling list 
    link = url
    img = driver.find_element_by_css_selector('#thumb img').get_attribute('src')
    brand = getBrand()
    title = driver.find_element_by_css_selector('.heading').text
    origin = getOrigin(title)
    type_ = '액상' if '액상' in title else '분말'
    prices = getPrice()
    rating = getRating()
    # option
    op_t, op_p = getOption()
    replyCount = getReplyCount()
    replyResult = getReplyResult()
    
    result = {
        'link':link,
        'img':img,
        'brand':brand,
        'title':title,
        'origin':origin,
        'type':type_,
        'price':prices,
        'rating':rating,
        'optionTitle':op_t,
        'optionPrice':op_p,
        'replyCount':replyCount,
        'replyResult':replyResult
    }    
    return result

In [914]:
def getBrand():
    try:
        brand = ''
        temp = driver.find_elements_by_css_selector('.prdc_detail_table tr')
        for i in range(len(temp)):
            if '브랜드' in temp[i].text:
                brand = temp[i].text.replace('브랜드 ', '')
                break
    except UnboundLocalError:
        None
    
    return brand

In [716]:
def getOrigin(title):
    for i in range(len(domestic)):
        origin = '국내' if domestic[i] in title else '해외'
    return origin

In [858]:
def getRating():
    try:
        rating = driver.find_element_by_css_selector('.selr_wrap strong').text
#         rating = int(r.replace('.',''))
    except NoSuchElementException:
        rating = 'rating 몰라'
    return rating

In [454]:
replyCount = 0
def getReplyCount():
    try:
        cnt = driver.find_element_by_css_selector('.notice_count').text
        replyCount = int(cnt[1:-1].replace(',',''))
    except NoSuchElementException:
        replyCount = 9999
    return replyCount

In [916]:
def getPrice():
    try:
        p = driver.find_element_by_css_selector('.sale_price')
        p = p.text
#         p = int(p.text.replace(',',''))
    
    except NoSuchElementException:
        p = 'price 몰라'
    except ValueError:
            p = int(p.text)
        
    return p

In [934]:
option_title = []
option_price =[]
def getOption():
    try:
        driver.find_element_by_css_selector('.option_select_box').click()

        options_title = driver.find_elements_by_css_selector('.op_list li .title')
        global option_title
        for title in options_title:
            if title.text is not '':
                option_title.append(title.text)

        options_price = driver.find_elements_by_css_selector('.op_list li .prdc_price em')
        global option_price
        for price in options_price:
            price = price.text
            if price is not '':
                option_price.append(price)

    except NoSuchElementException:
        None
    except UnboundLocalError:
        None
    except ValueError:
        None
        
    return option_title, option_price 

---

### Insert data into mongoDB

In [866]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

In [867]:
elevenst_db = client.get_database('elevenst_db')

In [868]:
# documents
milkPowder_collection = elevenst_db.get_collection('Milkpowder')

In [942]:
milkPowder_collection.insert_many([
    milkPowder_list[i] for i in range(len(milkPowder_list))
])

---

### Reply 
- 상품페이지에서 review 있는 곳으로 가는 부분이 iframe 처리되어있었음
- summ_txt( 자식 태그 )
- reviewContTxt ( 자식 태그 )

In [710]:
def getReplyResult():
    try:
        driver.switch_to_frame('ifrmReview')
        soup = bs(driver.page_source, 'lxml')
        replyResult = []
        page_num = 1

        while len(replyResult) < 20 :      
            # 페이지 이동 
            driver.find_element_by_css_selector('#paging_page').click()
            reply_list = driver.find_element_by_css_selector('.s_paging_v2 a')
            reply_page = reply_list.get_attribute('href')

            reply_page.replace('page=2','page={0}'.format(page_num))

            reply_1 = soup.select('.summ_conts .summ_txt')
            for r in reply_1:
                r = r.get_text()
                r = r[11:].rstrip()
                replyResult.append(r)

            reply_2 = soup.select('.summ_conts #reviewContTxt')
            for r in reply_2:
                r = r.get_text()
                r = r[11:].rstrip()
                replyResult.append(r)

            page_num += 1
    except NoSuchElementException:
        replyResult = None
    except NoSuchFrameException:
        replyResult = None
    
            
    return replyResult 

In [368]:
s = getReplyResult()

In [174]:
s

['남양꺼 먹이다가 요즘 사기도 힘들고 베비언스로 갈아타려구요..아기 먹이는건데 어찌 사는걸 이리 힘들게 해놨는지 이제 남양은 안살랍니다 아기꺼라 쉽게 못바꿨는데...잘 먹어주면 이걸로 ...쿠폰할인으로 싸게 샀어요',
 '친구집 갔다가 하나 아기에게 먹여봤는데 잘 먹더라구요. 할인해서 저도 한박스 샀네여. 유통기한도 길고 외출할때도 편하고. 울 아기 새벽에도 수유 하는 아기라 새벽에 분유 안타고 그냥 따서 줄려구요.',
 '아기가잘먹고 가지고다니기편하고 여러모로편리하지만 돌이되어가는 아기에겐 양이많아 버리고먹이거나 다른 젖병에 부어서 양을 조절해야하는게 아쉽네요',
 '막 던지셨어요?? 박스완전 다찌그러져서 걱정했는데ㅡㅡ 역시나 분유가 찌그러졌네요.. 애가먹는거라고 적혀있어도 택배기사님들 더운날 고생하는건 알겠지만..왜던지나 모르겟네요!!!!!화나네요!!!!!!’ㅜㅜ 다른데보다 저렴해서 삿더니 유통기한도짧고 고마 빨리먹고 구매하던데서 돈더주고 사는게 낫긋네요ㅡㅡ',
 '애기 둘키우는맘인데 액상분유는처음먹여봐요 가루분유는 외출시 번거롭고챙겨야할것도많은데 액상분유는 음료수처럼필요한만큼 몇개챙겨서 가방에넣고나기만하면되니 시간줄이고 챙겨야할것줄이고 암튼간편하고 좋아요 애기가 안먹어본거라 거부하면어쩌나 안먹으면어쩌나？x는데 다행히잘먹네요ㅎㅎ',
 '외출시 액상분유만큼 편한 제품이 없어서 구입했어요~~ 어른들은 이 제품보더니 요즘 참 세상 좋아졌다며 웃으시네요^^ 중복쿠폰 적용해서 저렴하게 구입하여 아주 만족합니다! 다행히 아이도 거부감없이 잘 먹어줘서 다행이네요^^',
 '편하게 하나씩 따서 먹이기 좋아요~ 유통기한은 12월 3일까지에요~ 폭염이라 걱정했는데 무사히 왔네요~ vip데이에 저렴히 득템했어요~~',
 '액상분유는 베비언스가 최고인듯ㅎㅎ 첫애도 이거 먹였어요~~ 간편하고 좋아요 살짝 달아서인지 아이가 정말 잘 먹네요~',
 '남양꺼 먹이다가 요즘 사기도 힘들고 베비언스로 갈아타려구요..아기 먹이는건데 어찌 사는걸 이리 힘들게 해놨는지 이제 남양은 안살랍니다

---

### Brand 
- 국내 / 해외 브랜드명 크롤링해서 List(domestic/overseas) 에 넣기 
- 제품명 split 해서 List(titleList) 에 넣어 놓은다음에 비교 

In [252]:
def getBrand(title):
    for title in title.split():
        if i in domestic:
            brand = i
            print(brand)

#### 국내/해외 브랜드 List 작업 

In [246]:
driver.get('http://www.11st.co.kr/browsing/DisplayCategory.tmall?method=getDisplayCategory2Depth&dispCtgrNo=1001550')

In [247]:
domestic_milk = driver.find_elements_by_css_selector('#areaBrandList .b_name')

In [248]:
domestic = [i.text for i in domestic_milk]

In [249]:
domestic[:5]

['남양유업', '일동후디스', '매일유업', '파스퇴르', '앱솔루트']

In [250]:
domestic.append('남양')

#### 제품명과 브랜드 리스트 비교 

In [374]:
t = '[11%쿠폰] 남양유업 액상분유 2/3단계 25입 1단계 24입'

In [265]:
for i in t.split():
    if i in domestic:
        brand = i
        print(brand)

일동후디스


In [379]:
for i in range(len(domestic)):
    brand = '국내' if domestic[i] in t else '해외'
        

### Run Crawling milk_powder

In [880]:
milkPowder_list = []

In [940]:
for i in range(47,99):
    result = getInfo_milkPowder(link[i])
    milkPowder_list.append(result)
    driver.implicitly_wait(1)

In [941]:
len(milkPowder_list)

98

In [939]:
milkPowder_list[45]

{'link': 'http://www.11st.co.kr/product/SellerProductDetail.tmall?method=getSellerProductDetail&prdNo=942000762&trTypeCd=22&trCtgrNo=895019',
 'img': 'http://i.011st.com/ex_t/R/400x400/1/85/0/src/pd/17/0/0/0/7/6/2/vZowa/942000762_B.jpg',
 'brand': '상품상세설명 참조',
 'title': '[[이베이비샵]]남양 임페리얼XO1~4단계/800g/사은품\n4시주문까지 당일발송',
 'origin': '국내',
 'type': '분말',
 'price': '22,000',
 'rating': '4.7',
 'optionTitle': ['XO 1단계(800g)',
  'XO 2단계(800g)',
  'XO 3단계(800g)',
  'XO 4단계(800g)'],
 'optionPrice': ['23,500', '23,500', '22,000', '22,000'],
 'replyCount': 733,
 'replyResult': ['좋아서 또 구매해요~ 비싼 분유를 가성비 좋게 먹을 수 있어서 좋고 배송도 잘 이루어집니다.',
  '하필 광복절이껴서.우체국이라 토욜도 쉬니 급히 필요해서 한통주문한건데 수욜인가 시켰는데 월욜에 받았네요.어쩔수없이 분유떨어져서 마트가서 스틱사고.다른택배는 토욜에 왔는데 여기만 월욜이 도착.분윤데 빨리보내주시지.그점만빼면.뭐...',
  '상품은 만족하지만 개념없는 택배기사때문에 불만 누르고 갑니다.택배사 바꾸셨음 좋겠어요;  앞으로도 여기서 구매하고 싶은데 그 택배기사 꼬락서니 보기 싫어서ㅜㅜ',
  '쿠폰다운받아서 많이 싸게 주문했어요~ 센스있게 손수건도 하나 넣어주셨네요ㅋㅋ 깨지는것도 아닌데 분유통 하나하나마다 뽁뽁이를 정성스럽게 다 싸주셨더라구요^^ 느므 감사하게 잘 받았습니다~♡',
  '제품은 찌그러지지도않구 포장두 잘 해주셨는데 택배기사님이